## Create PyTorch DataLoader

Maybe gonna be helpful for someone :)

In [ ]:
import os
import pandas as pd
import numpy as np

from skimage import io
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

TRAIN_CSV = '/kaggle/input/petfinder-pawpularity-score/train.csv'
TRAIN_IMAGES_DIR = '/kaggle/input/petfinder-pawpularity-score/train'


In [ ]:
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
])

train_csv = pd.read_csv(TRAIN_CSV)[['Id', 'Pawpularity']]
train_csv['Id'] = train_csv['Id'] + '.jpg'

train_csv.head()


In [ ]:
# a few modify default class
class CustomDataset(Dataset):
    def __init__(self, df_labels, root, transform=None):
        self.labels = df_labels
        self.root = root
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img_path = os.path.join(self.root, self.labels.iloc[index, 0])
        image = io.imread(img_path)
        label = torch.tensor(int(self.labels.iloc[index, 1]))

        if self.transform:
            image = self.transform(Image.fromarray(image))

        return (image, label)


In [ ]:
dataset = CustomDataset(df_labels=train_csv, root=TRAIN_IMAGES_DIR, transform=transform)
loader = DataLoader(dataset, batch_size=10, shuffle=True, num_workers=0)

In [ ]:
images, labels = next(iter(loader))